In [1]:
!pip uninstall tensorflow
!pip install tensorflow-gpu

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 516.2MB 29kB/s 


In [2]:
import tensorflow as tf
from pathlib import Path
print(tf.__version__)
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import pdb
from google.colab import drive
drive.mount('/content/drive')

2.2.0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# data_folder = Path('INSERT YOUR PATH HERE')
data_folder = Path('/content/drive/My Drive/Mary Jane Inc./Data')
player_data = data_folder / 'Player Stats'
team_data = data_folder / 'Team Stats'


In [4]:
years = ['09','10','11','12','13','14','15','16','17','18','19']
for i in tqdm(range(len(years) - 1)):
  if i == 0:
    df_p = pd.read_excel(player_data / f'NBA-20{years[i]}-20{years[i + 1]}-Player-BoxScore-Dataset.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-PLAYER')
    df_t = pd.read_excel(team_data / f'20{years[i]}-20{years[i + 1]}_NBA_Box_Score_Team-Stats.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-TEAM')
  else:
    temp_p = pd.read_excel(player_data / f'NBA-20{years[i]}-20{years[i + 1]}-Player-BoxScore-Dataset.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-PLAYER')
    temp_t = pd.read_excel(team_data / f'20{years[i]}-20{years[i + 1]}_NBA_Box_Score_Team-Stats.xlsx',sheet_name=f'NBA-20{years[i]}-{years[i + 1]}-TEAM')
    df_p = df_p.append(temp_p)
    df_t = df_t.append(temp_t)

100%|██████████| 10/10 [01:10<00:00,  7.06s/it]


In [0]:
def pre_process_player(player_frame):
  player_frame = player_frame.drop(['DATASET', 'PLAYER \nFULL NAME', 'OWN \nTEAM', 'OPPONENT \nTEAM'],axis=1)
  player_frame = player_frame.rename(columns={'VENUE\n(R/H)':'VENUE','STARTER\n(Y/N)':'STARTER','USAGE \nRATE (%)':'USAGE','DAYS\nREST':'REST','GAME-ID':'G_ID','PLAYER-ID':'P_ID'})
  starter_int = {start:int(i) for i, start in enumerate(['Y', 'N'])}
  venue_int = {ven:int(i) for i, ven in enumerate(['H','R'])}
  position_int = {pos:int(i) for i, pos in enumerate(['F', 'C', 'G', 'C-F', 'G-F', 'F-G', 'F-C', 'SF', 'PF', 'PG', 'SG', 'OTHER'])}
  rest_int = {'3+':int(666)}
  player_frame = player_frame.replace({'STARTER':starter_int,'VENUE':venue_int,'POSITION':position_int,'REST':rest_int})
  player_frame['POSITION'] = player_frame['POSITION'].fillna(11.0)
  player_frame['POSITION'] = player_frame['POSITION'].astype('int64')
  player_frame['REST'] = player_frame['REST'].astype('int64')
  player_frame['DATE'] = pd.to_datetime(player_frame['DATE'],format='%m/%d/%Y')
  return player_frame
df_p = pre_process_player(df_new)

In [0]:
df_p.columns
df_p = df_new

In [30]:
num_game_history = 3
cols = ['G_ID', 'DATE', 'P_ID', 'POSITION', 'VENUE', 'STARTER', 'MIN', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'OR', 'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS', 'USAGE', 'REST',  'g1_G_ID', 'g1_DATE', 'g1_P_ID', 'g1_POSITION', 'g1_VENUE', 'g1_STARTER', 'g1_MIN', 'g1_FG', 'g1_FGA', 'g1_3P', 'g1_3PA', 'g1_FT', 'g1_FTA', 'g1_OR', 'g1_DR', 'g1_TOT', 'g1_A', 'g1_PF', 'g1_ST', 'g1_TO', 'g1_BL', 'g1_PTS', 'g1_USAGE', 'g1_REST', 'g2_G_ID', 'g2_DATE', 'g2_P_ID', 'g2_POSITION', 'g2_VENUE', 'g2_STARTER', 'g2_MIN', 'g2_FG', 'g2_FGA', 'g2_3P', 'g2_3PA', 'g2_FT', 'g2_FTA', 'g2_OR', 'g2_DR', 'g2_TOT', 'g2_A', 'g2_PF', 'g2_ST', 'g2_TO', 'g2_BL', 'g2_PTS', 'g2_USAGE', 'g2_REST', 'g3_G_ID', 'g3_DATE', 'g3_P_ID', 'g3_POSITION', 'g3_VENUE', 'g3_STARTER', 'g3_MIN', 'g3_FG', 'g3_FGA', 'g3_3P', 'g3_3PA', 'g3_FT', 'g3_FTA', 'g3_OR', 'g3_DR', 'g3_TOT', 'g3_A', 'g3_PF', 'g3_ST', 'g3_TO', 'g3_BL', 'g3_PTS', 'g3_USAGE', 'g3_REST']
dataset = []
for id in tqdm(df_p['P_ID'].unique()):
  dis_person = df_p[df_p['P_ID'] == id]
  for index, row in dis_person.iterrows():
    prev_games = dis_person[dis_person['DATE'] <= row['DATE']].sort_values('DATE',ascending=False)
    # prev_games = prev_games.drop(columns=['G_ID','P_ID'])
    if len(prev_games) >= num_game_history + 1:
      prev_games = prev_games.nlargest(num_game_history + 1, 'DATE').values.flatten()
      dataset.append(prev_games)
      





  0%|          | 0/1238 [00:00<?, ?it/s]


  0%|          | 1/1238 [00:02<48:51,  2.37s/it]


  0%|          | 2/1238 [00:04<44:57,  2.18s/it]


  0%|          | 3/1238 [00:06<44:30,  2.16s/it]


  0%|          | 4/1238 [00:07<41:53,  2.04s/it]


  0%|          | 5/1238 [00:10<44:45,  2.18s/it]


  0%|          | 6/1238 [00:10<34:13,  1.67s/it]


  1%|          | 7/1238 [00:11<29:25,  1.43s/it]


  1%|          | 8/1238 [00:12<24:48,  1.21s/it]


  1%|          | 9/1238 [00:13<20:30,  1.00s/it]


  1%|          | 10/1238 [00:16<36:02,  1.76s/it]


  1%|          | 11/1238 [00:17<33:09,  1.62s/it]


  1%|          | 12/1238 [00:18<25:39,  1.26s/it]


  1%|          | 13/1238 [00:19<22:58,  1.13s/it]


  1%|          | 14/1238 [00:20<26:37,  1.30s/it]


  1%|          | 15/1238 [00:21<22:09,  1.09s/it]


  1%|▏         | 16/1238 [00:22<20:18,  1.00it/s]


  1%|▏         | 17/1238 [00:24<25:35,  1.26s/it]


  1%|▏         | 18/1238 [00:24<21:09,  1.04s/it]


  2%|▏         | 19/1238 [0

In [31]:
final_dataset = np.array(dataset)
print(final_dataset.shape)
df = pd.DataFrame(final_dataset)
df.to_csv(data_folder / 'NBA_FULL_PLAYER.csv',index=False)

(266560, 96)


In [0]:
data = pd.read_csv(data_folder / 'NBA_FULL_PLAYER.csv', delimiter=',')
data['1'] = data['1'].apply(lambda x: int(x[2: 4]))
data['25'] = data['25'].apply(lambda x: int(x[2: 4]))
data['49'] = data['49'].apply(lambda x: int(x[2: 4]))
data['73'] = data['73'].apply(lambda x: int(x[2: 4]))
# data = data.to_numpy()
# data = data.values

In [122]:
data.values.shape
targets = data.values[:,6:22]
print("Target Shape:", targets.shape )
features = data.values[:,:]
features = np.delete(features,np.s_[6:22],1)
features = np.delete(features,0,1)
features = np.delete(features,1,1)
features = np.delete(features,6,1)
features = np.delete(features,7,1)
features = np.delete(features,28,1)
features = np.delete(features,29,1)
features = np.delete(features,50,1)
features = np.delete(features,51,1)
features.shape
print("Features Shape:", features.shape )

Target Shape: (266560, 16)
Features Shape: (266560, 72)


In [62]:
# i = 1
# build_row = np.array(data[i][27:48])
# build_row = np.vstack((build_row,np.array(data[2][27:48])))
# build_row

array([[0, 0, 0, 36.88, 10, 14, 3, 5, 4, 6, 1, 3, 4, 2, 2, 2, 2, 0, 27,
        11.285270343630714, 4],
       [0, 1, 0, 30.67, 8, 13, 1, 3, 4, 4, 1, 7, 8, 4, 2, 2, 3, 0, 21,
        28.582473966203537, 3]], dtype=object)

In [46]:
# inputs = np.array()
# outputs = np.array()
# for i in len(data):
#   # Date, position
#   build_row = np.array(data[i][1],data[i][3:6],data[i][22:24], )
#   inputs.append(data[i][:]

20900040

In [148]:
features.shape

(266560, 72)

In [146]:
input_dim = features.shape[1]
latent_dim = 1
model = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(input_dim,)),
        tf.keras.layers.Dense(input_dim, activation='relu'),
        tf.keras.layers.Dense(int(input_dim - ((input_dim - latent_dim)/2)), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16),
    ]
)
model.summary()
optimizer = tf.keras.optimizers.Adam(1e-4)
model.compile(optimizer,'mse')

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_80 (Dense)             (None, 72)                5256      
_________________________________________________________________
dense_81 (Dense)             (None, 36)                2628      
_________________________________________________________________
flatten_26 (Flatten)         (None, 36)                0         
_________________________________________________________________
dense_82 (Dense)             (None, 16)                592       
Total params: 8,476
Trainable params: 8,476
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
6664/6664 [==============================] - 10s 1ms/step - loss: 16.3498 - val_loss: 8.3062
Epoch 2/5
6664/6664 [==============================] - 10s 1ms/step - loss: 8.6358 - val_loss: 7.7280
Epoch 3/5
6664/6664 [=======================

In [0]:
model.fit(x=features, y=targets, epochs=5, validation_split=0.20, batch_size=32)

In [0]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, input_dim, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(input_dim,)),
            tf.keras.layers.Dense(input_dim, activation='relu'),
            tf.keras.layers.Dense(int(input_dim - ((input_dim - latent_dim)/2)), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=input_dim, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(input_dim,)),
            tf.keras.layers.Dense(input_dim, activation='relu'),
            # tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same',activation='relu'),
            # tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same',activation='relu'),
            # # No activation
            # tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [0]:
cvae = CVAE(72, 10)

In [150]:
type(cvae)

__main__.CVAE

In [0]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  pdb.set_trace()
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [164]:
train_step(cvae, features, optimizer)

> /tmp/tmpnxpwu0rc.py(14)tf__compute_loss()
-> cross_ent = ag__.converted_call(tf.nn.sigmoid_cross_entropy_with_logits, (), dict(logits=x_logit, labels=x), fscope)
(Pdb) x_logit
<tf.Tensor 'sequential_30/reshape_4/Reshape:0' shape=(266560, 72) dtype=float32>
(Pdb) c


TypeError: ignored